In [ ]:
#*** INITIALISATION ***#

import os
from pathlib import Path
from dotenv import load_dotenv
import datetime

load_dotenv()

# Timestamp
date_now = datetime.date.today().isoformat()
datetime_now = datetime.datetime.now().isoformat()

print(os.path.abspath(os.curdir))

In [ ]:
#*** TÉLÉCHARGEMENT ***#

import pandas as pd
from requests import get

# decp_augmente_valides_file: Path = Path(f'data/decp_augmente_valides_{date_now}.csv')
decp_augmente_valides_file: Path = Path(f'data/decp_augmente_valides_2024-04-12.csv')


if not(os.path.exists(decp_augmente_valides_file)):
    request = get(os.getenv('DECP_ENRICHIES_VALIDES_URL'))
    with open(decp_augmente_valides_file, 'wb') as file:
        file.write(request.content)
else:
    print("DECP d'aujourd'hui déjà téléchargées")

df: pd.DataFrame = pd.read_csv(decp_augmente_valides_file, sep=";", dtype='object', index_col=None)   


In [ ]:
#*** ANALYSE DE BASE ***#

df.info(verbose=True)
# obsolete number of rows 994123
# valid number of rows 837115

In [ ]:
from numpy import NaN

df.replace([NaN, None], "", inplace=True, regex=False)
# df[['id', 'datePublicationDonnees']].loc[df['datePublicationDonnees'].str.contains("September")]

In [ ]:
#*** REDRESSEMENT ***#

# Dates

columns_date = ["datePublicationDonnees", "dateNotification"]

date_replacements = {
    # ID marché invalide et SIRET de l'acheteur
    "0002-11-30": "",
    "September, 16 2021 00:00:00": "2021-09-16", #20007695800012 19830766200017 (plein !)
    "16 2021 00:00:00": "",
    "0222-04-29": "2022-04-29", # 202201L0100
    "0021-12-05": "2022-12-05", # 20222022/1400
    "0001-06-21": "", #0000000000000000 21850109600018
    "0019-10-18": "", #0000000000000000 34857909500012
    "5021-02-18": "2021-02-18", #20213051200 21590015000016   
    "2921-11-19": "", # 20220057201 20005226400013
    "0022-04-29": "2022-04-29", # 2022AOO-GASL0100 25640454200035   
}

for col in columns_date:
    df[col] = df[col].replace(date_replacements, regex=False)


# Nombres

df['dureeMois'] = df['dureeMois'].replace('', NaN)
df['montant'] = df['montant'].replace('', NaN)

# Identifiants de marchés

id_replacements = {
    "[,\./]": "_"
}

df["id"] = df["id"].replace(id_replacements, regex=True)




In [ ]:

# df["datePublicationDonnees"].loc[df["datePublicationDonnees"] == '2921-11-19']
# df[['id', 'acheteur.id', 'dateNotification', 'datePublicationDonnees']].loc[df['acheteur.nom'] == None]
# df.loc[2067]

In [ ]:
#***  TYPES DE DONNÉES ***#

numeric_dtypes = {
    "dureeMois": "Int64", # contrairement à int64, Int64 autorise les valeurs nulles https://pandas.pydata.org/docs/user_guide/integer_na.html
    "montant": "float64"
}

for column in numeric_dtypes:
    df[column] = df[column].astype(numeric_dtypes[column])


date_dtypes = ["datePublicationDonnees", "dateNotification"]

for column in date_dtypes:
    df[column] = pd.to_datetime(df[column], format='mixed', dayfirst=True)


df.info(verbose=True)

In [ ]:
#***  ANALYSES AVANCÉES ***#

# Les sources d'où proviennent les données
sources = df["source"].unique()

for source in sources:
    print(f"""
# {source}
Nombre de marchés : {df[["source"]].loc[df["source"]==source].index.size}
Nombre d'acheteurs uniques : {len(df[["acheteur.id", "source"]].loc[df["source"]==source]['acheteur.id'].unique())}
""")

In [ ]:
# VERS LE FORMAT DECP-TABLE-SCHEMA #

# Explosion des champs titulaires sur plusieurs lignes

df['titulaire.id'] = [[] for r in range(len(df))]
df['titulaire.denominationSociale'] = [[] for r in range(len(df))]
df['titulaire.typeIdentifiant'] = [[] for r in range(len(df))]

for num in range(1, 4):
    mask = df[f'titulaire_id_{num}'] != ""
    df.loc[mask, 'titulaire.id'] += df.loc[mask, f'titulaire_id_{num}'].apply(lambda x: [x])
    df.loc[mask, 'titulaire.denominationSociale'] += df.loc[mask, f'titulaire_denominationSociale_{num}'].apply(lambda x: [x])
    df.loc[mask, 'titulaire.typeIdentifiant'] += df.loc[mask, f'titulaire_typeIdentifiant_{num}'].apply(lambda x: [x])

df = df.explode(['titulaire.id', 'titulaire.denominationSociale','titulaire.typeIdentifiant'],ignore_index=True)

In [ ]:
# Ajout colonnes manquantes

df['uid'] = df['acheteur.id'] + df['id']
df['donneesActuelles'] = "" # TODO
df['anomalies'] = "" # TODO

df.info()


In [ ]:
#*** ENREGISTREMENT AU FORMAT CSV ET PARQUET ***#

import shutil

distdir = Path('./dist')
if (os.path.exists(distdir)):
    shutil.rmtree('dist')
os.mkdir('dist')
os.chdir(distdir)

# CSV

# Schéma cible
df_standard = pd.read_csv("https://raw.githubusercontent.com/ColinMaudry/decp-table-schema/main/exemples/exemple-valide.csv", index_col=None)

cible_colonnes = df_standard.columns
df = df[cible_colonnes]
# df_sans_titulaires = df.drop(columns=['titulaire.id', 'titulaire.denominationSociale', 'titulaire.typeIdentifiant'])

df.to_csv('decp.csv', index=None)
# df_sans_titulaires.to_csv('decp-sans-titulaires.csv', index=None)

df.to_parquet('decp.parquet', index=None)


In [ ]:
# VALIDATION #

from tableschema import Table, CastError

table = Table('decp.csv', schema='https://raw.githubusercontent.com/ColinMaudry/decp-table-schema/main/schema.json')
try:
    table.read(limit=50)
except CastError as exception:
    print(exception.errors)

In [ ]:
# DATA PACKAGE #

from frictionless import Package, Resource, Pipeline, steps

decp_resource =  Resource(path='decp.csv')

# Cette méthode détecte les caractéristiques du CSV et tente de deviner les datatypes
decp_resource.infer()

pipeline = Pipeline(
    steps=[
        steps.field_update(name="acheteur.id", descriptor={"type": "string"}),
        steps.field_update(name="acheteur.nom", descriptor={"type": "string"}),
        steps.field_update(name="titulaire.id", descriptor={"type": "string"}),
    ])

decp_resource = decp_resource.transform(pipeline)


package = Package(
    name='decp',
    title='DECP CSV',
    description='Données essentielles de la commande publique (FR) au format CSV.',
    resources=[decp_resource],
    # it's possible to provide all the official properties like homepage, version, etc
)

package.to_json('datapackage.json')

In [ ]:
#*** SQLITE ***#

from datapackage_to_datasette import datapackage_to_datasette

if os.path.exists('decp.sqlite'):
    os.remove('decp.sqlite')
 
datapackage_to_datasette( 
            dbname='decp.sqlite',
            data_package='datapackage.json',
            metadata_filename='datasette_metadata.json',
            write_mode='replace'
        )

In [ ]:
#*** PUBLICATION SUR DATA.GOUV.FR ***#

import requests
import json

# TODO: à tester, attention le téléchargement du CSV a l'air d'avoir lieu malgré sa présence en local

api_key = os.getenv('DATAGOUVFR_API_KEY')

api="https://www.data.gouv.fr/api/1"
dataset_id="608c055b35eb4e6ee20eb325"
resource_id_decp="8587fe77-fb31-4155-8753-f6a3c5e0f5c9"
# resource_id_sans_titulaires="834c14dd-037c-4825-958d-0a841c4777ae"
resource_id_datapackage="65194f6f-e273-4067-8075-56f072d56baf"
resource_id_sqlite="c6b08d03-7aa4-4132-b5b2-fd76633feecc"

def update_resource(api, dataset_id, resource_id, file_path, api_key):
    url = f"{api}/datasets/{dataset_id}/resources/{resource_id}/upload/"
    headers = {
        "X-API-KEY": api_key
    }
    files = {
        "file": open(file_path, 'rb')
    }
    response = requests.post(url, files=files, headers=headers)
    return response.json()

print("Mise à jour de decp.csv...")
print(json.dumps(update_resource(api, dataset_id, resource_id_decp, "decp.csv", api_key), indent=4))

# print("\nMise à jour de decp-sans-titulaires.csv...")
# print(json.dumps(update_resource(api, dataset_id, resource_id_sans_titulaires, "decp-sans-titulaires.csv", api_key), indent=4))

print("\nMise à jour de datapackage.json...")
print(json.dumps(update_resource(api, dataset_id, resource_id_datapackage, "datapackage.json", api_key), indent=4))

print("\nMise à jour de decp.sqlite...")
print(json.dumps(update_resource(api, dataset_id, resource_id_sqlite, "decp.sqlite", api_key), indent=4))